This is a tutorial notebook on object-oriented software design, prepared for the Foundations of Software Engineering (FSE v2020.1, https://github.com/adasegroup/FSE2020_seminars) at Skoltech (http://skoltech.ru).

Copyright 2020 by Alexey Artemov and ADASE Lab. 

# FSE-08: Object-Oriented Software Design: Creational Design Patterns

## 1. Singleton: disabling instantiation of more than one object

In some instances, we would like to disable instantiation of more than 1 object of a particular class. 

**Why would we like to have only 1 object of a class?**

Acceptable reasons to use a Singleton include:

 - Access to a "heavy" shared resource (e.g. a database), preferred that an access to the resource will be requested from multiple, disparate parts of the system.
 - "Logging class rationale": heavy re-use of the same class instance by lots of callers 
 > A Singleton can be used instead of a single instance of a class because a logging class usually needs to be used over and over again ad nauseam by every class in a project. If every class uses this logging class, dependency injection becomes cumbersome. ([When should I use the singleton?](https://stackoverflow.com/questions/228164/on-design-patterns-when-should-i-use-the-singleton))
 
 
 
**Example implementation:**

(more variants at [Creating a singleton in Python](https://stackoverflow.com/questions/6760685/creating-a-singleton-in-python))

```python
class Singleton:
    __instance = None
    def __new__(cls, *args):
        if cls.__instance is None:
            cls.__instance = object.__new__(cls, *args)
        return cls.__instance
```

**Task 1:** 
1. Explain where can a Singleton pattern be used within the virus spread modelling system.
2. Implement a Singleton pattern with the `DepartmentOfHealth` and `GlobalContext` classes.

#### Solution to task 1:

In [178]:
# YOUR CODE HERE

In [182]:
d1 = DepartmentOfHealth([])

In [183]:
d2 = DepartmentOfHealth([])

In [184]:
d1 is d2

True

In [190]:
# YOUR CODE HERE

---

## 2. Factory method: single point of access to creation of objects

If you're having a number of object subclasses with a shared superclass, it is convenient to provide a single point of access to generating instances of these classes.

**Why would we like to have a function rather than calling constructors explicitly?**

Acceptable reasons to use a Factory method include:

 - Don't need (or know how) the exact class of the object to be instantiated, or, as a client, would like to delegate the decision about instantiation to a different class.
 
 
**Example implementation:**

```python
class Person:
    def get_name(self):
        pass

class Villager(Person):
    def get_name(self):
        return "Village Person"
        
class CityPerson(Person):
    def get_name(self):
        return "City Person"

class PersonType(Enum):
    Rural = 1
    Urban = 2
    
def get_person(person_type: PersonType):
    if PersonType.Rural == person_type:
        return Villager()
    elif PersonType.Urban == person_type:
        return CityPerson()
    else:
        raise ValueError()
```

**Task 2:** 
1. Explain where can a Factory Method pattern be used within the virus spread modelling system.
2. Create a virus factory method and implement the basic virus functionality.

<img src="images/Factory Method Infectable Class.svg">

#### Solution to task 2:

In [185]:
# YOUR CODE HERE

---

## 3. Abstract factory

Suppose you want to be able to create instances of multiple sibling classes and use their genetic interfaces. 

<img width=640 src="images/Abstract_factory.svg">

**Why would we like to have a function rather than calling constructors explicitly?**

Acceptable reasons to use an Abstract Factory method include ([Why do we need Abstract factory design pattern?](https://stackoverflow.com/questions/2280170/why-do-we-need-abstract-factory-design-pattern)):

 - Any place where you need a run-time value to construct a particular dependency
 
 
**Example implementation:**

```python
from abc import ABC, abstractmethod
from sys import platform


class Button(ABC):
    @abstractmethod
    def paint(self):
        pass

class LinuxButton(Button):
    def paint(self):
        return 'Render a button in a Linux style'

class WindowsButton(Button):
    def paint(self):
        return 'Render a button in a Windows style'

class MacOSButton(Button):
    def paint(self):
        return 'Render a button in a MacOS style'

class GUIFactory(ABC):
    @abstractmethod
    def create_button(self):
        pass

class LinuxFactory(GUIFactory):
    def create_button(self):
        return LinuxButton()

class WindowsFactory(GUIFactory):
    def create_button(self):
        return WindowsButton()

class MacOSFactory(GUIFactory):
    def create_button(self):
        return MacOSButton()

if platform == 'linux':
    factory = LinuxFactory()
elif platform == 'darwin':
    factory = MacOSFactory()
elif platform == 'win32':
    factory = WindowsFactory()
else:
    raise NotImplementedError(
        f'Not implemented for your platform: {platform}'
    )

button = factory.create_button()
result = button.paint()
print(result)
```

#### Task 3

1. Explain where can an Abstract Factory pattern be used within the virus spread modelling system.
2. Create `AbstractPersonFactory` class to provide `get_person` interface, and subclass it to `DefaultPersonFactory` and `CommunityPersonFactory` yielding the respective `DefaultPerson` and `CommunityPerson` instances.

<img src="images/Abstract Person Factory Class.svg"/>

#### Solution to Task 3

In [188]:
# YOUR CODE HERE

In [195]:
# YOUR CODE HERE

In [196]:
def create_persons(min_j, max_j, min_i, max_i, n_persons):
    factory_params = (min_j, max_j, min_i, max_i)
    
    default_factory = DefaultPersonFactory(*factory_params)
    community_factory = CommunityPersonFactory(*factory_params, community_position=(50, 50))

    n_default_persons = int(n_persons * 0.75)
    n_community_persons = n_persons - n_default_persons

    persons = []
    for i in range(n_default_persons):
        persons.append(default_factory.get_person())
        
    for i in range(n_community_persons):
        persons.append(community_factory.get_person())

    return persons

In [198]:
create_persons(0, 99, 0, 99, 10)

## 4. Builder: parameterise the construction process of a complex object

If 
 - you're having an object (Client, Director) that has an idea of WHAT the contents of SOMETHING should be,
 - but you do not want to concern this object with the actual intantiation of SOMETHING,

then
 - you can delegate the creation to another object (Service, Builder)
 - that has an idea of HOW to actually instantiate SOMETHING.

<img width=640 src="images/Builder_UML_class_diagram.svg"/>

**Why would we like to have a partially parameterized Builder rather than calling constructors explicitly?**

Acceptable reasons to use a Builder pattern include ([What are the advantages of Builder Pattern of GoF?](https://softwareengineering.stackexchange.com/questions/345688/what-are-the-advantages-of-builder-pattern-of-gof)):

 - The object cannot be constructed in one call. This might be the case when the product is immutable, or contains complex object graphs e.g. with circular references.
 - You have one build process that should build many different kinds of products. 
 
**Example implementation:**

```python
class Movie:
    def __init__(self, actors, scenes, screenplay):
        self._actors = actors
        self._scenes = scenes
        self._screenplay = screenplay


class MovieDirector:
    '''Someone who knows the parameterization of an object to be constructed, 
    i.e. knows WHAT to build.'''
    def __init__(self, builder):
        self._builder = builder

    def create_movie(self):
        self._builder._screenplay = [
            'The girl disappears mysteriously in SF.',
            'A brave detective is on the journey to find the girl.'
        ]
        self._builder._actors = ['Natalie Portman', 'Brad Pitt']


class MovieOperator:
    '''Someone who knows HOW to build but expects more information on specific parameterization.'''
    def __init__(self):
        self._scenes = ['Golden Gate Bridge', 'San Francisco suburbs']
        self._screenplay = None
        self._actors = None
        
    def shoot_movie(self):
        movie = Movie(self._actors, self._scenes, self._screenplay)
        return movie
    
    
# Usage example:
    operator = MovieOperator()
    director = MovieDirector(operator)
    director.create_movie()
    movie = operator.shoot_movie()
```

#### Task 4:

Implement a Builder pattern for selecting a type of drugs for treating the patient in a Hospital.
 1. Create a interface `Drug` with a method `apply` acting on a `Person` instance
 2. According to the following class diagram, implement the entire class tree. _Note:_ `AntipyreticDrug` and `RehydrationDrug` and `AntivirusDrug` provide no real extension of the `Drug` interface. 
 3. For `AntipyreticDrug`, implement concrete subclasses `Aspirin`, `Ibuprofen`;
 for `RehydrationDrug`, implement concrete subclasses `Glucose`, `Rehydron`;
 for `AntivirusDrug`, implement concrete subclasses `Placebo`, `AntivirusSeasonalFlu`, `AntivirusSARSCoV2`, `AntivirusCholera`.

<img src="images/Drug Class.svg"/>

In [92]:
# YOUR CODE HERE

#### Task 5: continue with builder implementation, create builder classes

Implement the class hierarchy for `DrugReposistory`, responsible for producing concrete drug sequences:
 1. Interface `DrugRepository` with methods depicted in the diagram. 
 2. Subclass `DrugRepository` with `CheapDrugRepository` and `ExpensiveDrugRepository` that are able to create different types of drugs.

<img src="images/DrugRepository Class.svg"/>

#### Solution to task 5

In [199]:
# YOUR CODE HERE

#### Task 6: continue with builder implementation, create director hierarchy / methods

We need to identify the patient's desease, create an appropriate treatment plan, and execute treatment:
 1. Create a factory method `get_prescription_method` that would create one of the treatment.
 2. Create a class hierarchy of `AbstractPrescriptor` (see diagram below) to invoke the creation of a specific complex treatment.

<img src="images/AbstractPrescriptor Class.svg"/>

In [203]:
# YOUR CODE HERE

In [204]:
class Hospital:
    def __init__(self, capacity, doctor, drug_repository):
        self.doctor = doctor
        self.drug_repository = drug_repository
        self.capacity = capacity
        self.patients = []
        self.tests = []

    def _treat_patient(self, patient):
        # 1. identify disease
#         if patient.virus is not None:
#             disease_type = patient.virus.get_type()
        prescription_method = get_prescription_method(disease_type, self.drug_repository)
        
        # 2. understand dose
        
        # 3. compose treatment
        prescription_drugs = prescription_method.create_prescription()
        
        # 4. apply treatment
        for drug in prescription_drugs:
            patient.take_drug(drug)
            drug.apply(patient)

    def treat_patients():
        for patient in self._patients:
            self._treat_patient(patient)

---

## 5. Prototype: clone the object instead of creating the new one

In some instances, creating a new object instance is best achieved by calling a method of the existing object instance.

**Why would we like to clone objects rather than instantiate them explicitly?**

Acceptable reasons to use a Prototype pattern include:

 - Not being able to copy some of the fields of the object (e.g. private fields)
 - The need to save resources during object instantiation (e.g. a database call)
 - The need to avoid lots of boilerplate code used to parameterise the object
 
**Example implementation without copy/deepcopy:**

```python
class Bacteria:
    def __init__(self, size=None, lifetime=None, other=None):
        if None is other:
            self._size = size
            self._lifetime = lifetime
        else:
            self._size = other._size
            self._lifetime = other._lifetime
    
    def clone(self) -> Bacteria:
        return Bacteria(self)

    
class PoisonousBacteria(Bacteria):
    def __init__(self, poison, other=None, *args):
        super().__init__(other=other, *args)
        if None is other:
            self._poison = poison
        else:
            self._poison = other._poison
    
    def clone(self) -> PoisonousBacteria:
        return PoisonousBacteria(self)


# Usage:
bacteria = PoisonousBacteria(poison='poison', size=0.1, lifetime=100)
other = bacteria.clone()
```


**Example implementation USING copy/deepcopy:**
```python
import copy


class Bacteria:
    def __init__(self, size, lifetime):
        self._size = size
        self._lifetime = lifetime
    
    def __copy__(self):
        # shallow copy each of the objects
        size = copy.copy(self._size)
        lifetime = copy.copy(self._lifetime)
        
        # shallow copy of ourselves into the newly allocated object
        new = self.__class__(size, lifetime)
        new.__dict__.update(self.__dict__)
        
        return new

    def __deepcopy__(self, memo={}):
        # shallow copy each of the objects
        size = copy.deepcopy(self._size, memo)
        lifetime = copy.deepcopy(self._lifetime, memo)
        
        # shallow copy of ourselves into the newly allocated object
        new = self.__class__(size, lifetime)
        new.__dict__ = copy.deepcopy(self.__dict__, memo)
        
        return new

    
# Usage:
bacteria = Bacteria(size=0.1, lifetime=100)
other = copy.deepcopy(bacteria)

```